# Bank Customer Churn Prediction (ANN) Keras Model

In [1]:
import numpy as np  
import pandas as pd 
import tensorflow as tf 

In [2]:
tf.__version__   # Tensorflow sürümünü kontrol ediyorum.

'2.20.0'

## Part 1 - Data Preprocessing [Veri setimizde her şey sayı olmalı bu yüzden france , male gibi özellikleri aşağıdaki yöntemler ile sayısal karşılık ataması yapmamız gerekiyor]

### Importing the dataset (Veri setini içe aktarma)





In [3]:
import pandas as pd

# "Churn_Modelling.csv" adlı dosya aslında bir Excel dosyası
dataset = pd.read_excel("Churn_Modelling.csv", engine="openpyxl")  # Data setimizi içe aktarıyoruz.

X = dataset.iloc[:, 3:-1].values # Buradaki x fonksiyonumuzun x değeri diye düşünebilriiz veri girdisi. ".iloc" Pandas fonksiyonu, DataFrame içindeki verilere endekslemek için kullanılır. [:, 3:-1] ifadesi, tüm satırları alır (:) ve sütunlarda 3. sütundan (indeks 3) başlayıp son sütundan bir öncekine kadar (-1 indeks) olan sütunları seçer. Çünkü ilk 3 sütun bizi modelimiz için gerekmiyor kullanıcının adı soyadı veya satır sayısı onun bankada kalıp kalmama durumuna etki etmez. Aslında sinir ağımız bunu kendisi de anlayabilir ancak eğitim sürecini hızlandırmak için bunu biz yapalım ve ".values" DataFrame'in değerlerini Numpy array olarak döndürür.
y = dataset.iloc[:, -1].values # Buradaki y değeri ise Veri setimizin en son satırlarında bulunan Exited değeri yani müşteri banka da kalacak mı kalmayacak mı ? yani f(x) = y değeridir. 1 ise müşteri bankadan ayrılmıştır 0 ise hala bankada kalıyor anlamına gelir.


In [4]:
print(dataset.shape)  # (10000, 14)
print(X.shape)        # (10000, 10)
print(y.shape)        # (10000,)

(10000, 14)
(10000, 10)
(10000,)


### Encoding categorical data

Label Encoding the "Gender" column

In [5]:
from sklearn.preprocessing import LabelEncoder # LabelEncoder sınıfını kullanarak kategorik verilerin (string veya obje olarak temsil edilen) sayısal değerlere dönüştürülmesini sağlar.
le = LabelEncoder() # Bu sınıftan bir "le" oluşturdum veri dönüşümlerini yapmak için kullanılacak yöntemlere sahip olur.
X[:, 2] = le.fit_transform(X[:, 2]) # X[:, 2] ifadesi, X veri setinin tüm satırlarını (:) ve 2. sütununu seçer. le.fit_transform(X[:, 2]) komutu ile veri setinin 2. sütununu sayısal değerlere dönüştürüp tekrar 2. sütuna yazar.

In [6]:
print(X) # Makine rastgele olarak kadına 0 ve erkeğe 1 değerlerini atamış.

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder # Ülke isimlerini 0 1 2 diye rastgele numaralandıramayız bu yüzden HotEncoder kullanmamız gerekiyor.
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') # Dönüşüm yapılacak sütunun indeksini belirtiyoruz "1". ct adında bir kısa bir örnek oluşturuyoruz.
X = np.array(ct.fit_transform(X)) # ct.fit_transform(X) yöntemi, X veri setini ColumnTransformer ile dönüştürür. Dönüşüm sonucu, X veri seti üzerine uygulanır ve dönüştürülmüş veriyi içeren bir Numpy array'i döndürür. np.array() fonksiyonu ile dönüşüm sonucu tekrar bir Numpy array'e dönüştürülür ve X değişkenine atılır.

In [8]:
print(X) # Fransa 1.0 0.0 0.0 , Almanya 0.0 1.0 0.0 , İspanya 0.0 0.0 1.0 diye kodlanmış # Şuan burada cinsiyet sayıları [0 ve 1] gözükmüyor ama sıkıntı yok o hala o array in içinde sadece dizi uzun olduğu için gözükmüyor.

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set (Veri setini train ve test olacak şekilde ikiye ayıracağız)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) # train_test_split fonksiyonu, dört değişken döndürür: X_train, X_test, y_train, ve y_test

### Feature Scaling (Özellik Ölçeklendirme)

In [10]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() # sc adında kısa bir kullanım örneği oluşturduk.
X_train = sc.fit_transform(X_train) # sc örneğini kullanarak fit_transform yöntemi, önce veriye fit (uygun hale getirme) işlemi yapar ve sonra veriyi dönüştürme (transform) işlemi uygular. X_train veri seti, StandardScaler ile fit edilir (yani ortalama ve standart sapma hesaplanır) ve dönüştürülür (standardizasyon yapılır).Bu işlem sonucunda, X_train içindeki her bir özellik (sütun) bağımsız olarak standart normal dağılıma göre ölçeklenmiş olur.
X_test = sc.transform(X_test) # transform yöntemi, veriyi dönüştürme (transform) işlemi uygular. X_test veri seti, daha önce fit edilen StandardScaler ile dönüştürülür (standardizasyon yapılır). Burada X_test verisi, eğitim verisi olan X_train üzerinde fit edilen standartlaştırma parametreleri (ortalama ve standart sapma) kullanılarak dönüştürülür.
# Test setinin eğitim setinden ayrı olduğunu unutmamak önemlidir; bu nedenle, test seti üzerinde yalnızca transform işlemi yapılır.

## Part 2 - Building the ANN

### Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential() # Oluşturduğumuz sinir ağı bir sıralı sınıfın nesnesi olacaktır. ancak grafik şeklinde değil BİR DİZİ KATMAN şeklindedir.
#  tf.keras.models 'u çağırdık çünkü bizim SIRALI SINIFI çağıracak olan fonksiyon budur.
# SONUÇ OLARAK ANN imizi bir dizi katman olarak başlatan bir "ann" değişkenini oluşturduk.
# sequential == sıralı demek sıralı sınıfımız budur. Bu sınıfın içinde kesinlikle ADD komutu olmalıdır buna dikkat edelim.
# Sequential modeli, katmanları sıralı olarak eklememizi sağlayan bir yapıdır.

# -- ÖNEMLİ --
# Yapay sinir ağlarında, giriş katmanı (input layer) genellikle açıkça eklenmez çünkü bu katman, model oluşturulurken girdi verileri input layer olarak kabul edilir zaten ekli varsayılır.
# Giriş katmanı, model oluşturulurken verilen ilk katman olarak kabul edilir.
# Dolayısıyla, Sequential modeli oluşturulurken ilk katman eklenirken aynı zamanda giriş katmanı olarak kabul edilir.

### Adding the input layer and the first hidden layer (İnput ve gizli katmanları eklemeye başlayalım)

In [12]:
#                                          SIĞ BİR SİNİR AĞI NASIL OLUŞTURULUR ?
ann.add(tf.keras.layers.Dense(units=8, activation='relu')) # Gizli katman ekledim.katman sayısını artırarak ağımızı daha da derinleştirebiliriz.
# ADD komutu ile herhangi bir şey bağladığımızda kullanırız ister hidden layer yani gizli katman olsun ister input layer olsun.
# Veri setimizdeki her sütun bir input nöron olacaktır.
# units=6: Bu parametre, katmandaki nöron sayısını belirtir. Burada, 6 adet nöron (veya birim) bulunan bir gizli katman ekleniyor.
# activation='relu': Bu parametre, katmanın aktivasyon fonksiyonunu belirtir. 'relu' (Rectified Linear Activation) fonksiyonu, gizli katmanlarda sıkça kullanılan ve Rectifier (Doğrultucu fonksiyon , defterde notu grafiği mevcut) aktivasyon fonksiyonudur. Bu fonksiyonun kod adı 'relu' olarak kullanılır.
# Bu fonksiyon, negatif girişlerde 0 çıktısı verirken, pozitif girişlerde girişi doğrudan geçirir.
# units=6 bu komut katmandaki nöron sayısını belirtir. Burada, 6 adet nöron (veya birim) bulunan bir GİZLİ KATMAN ekleniyor.
# Dense komutu katmanları birbirine bağlar önceki katmanlarla eklediğimiz katmanların bağlantısını yapar.

# -- ÖNEMLİ --
# Bir katmanda kaç nöron olması gerektiğini nasıl belirleriz ?
# Bu tamamen tecrübeye dayanır bunun için geliştirilmiş bir formül yoktur.
# Şimdilik bunu deneyerek ayarlamamız gerekiyor hangi sonucun doğruluk değeri daha yüksek ise o nöron sayısını seçeceğiz.
# Ancak bu sayılar çok uçuk kaçık sayılar olmaması gerekiyor. Tabii ki veri setinize de bağlı ancak yine de büyük sayılar olmaz.

### Adding the others hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu')) # Bir diğer gizli katman ekledim.
# İstersem projeye göre bu değerleri değiştirebilirim ya da nöron sayını modelinin doğruluk yüzdesini artırmak için güncelleyebilirim.

In [14]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu')) # Bir diğer gizli katman ekledim.

### Adding the output layer



In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Sonuçta yeni bir katman eklediğimiz için "ann.add(tf.keras.layers.Dense" buraya kadar olan kısım aynı
# 2 sınıf sonucu kullandığımız için tek çıkış nöranu tercih ettik yani "units=1" oldu. genelden evet hayır doğru yanlış hasta sağlıklı gibi çift sonuçlu veri setlerinde tek çıkış nöronu kullanılır.
# İkiden çok sınıf ve İkiden çok çıktı değeri olan sinir ağlarında daha fazla nöron sayısı kullanabiliriz.
# Sigmoid fonksiyonu defterde de anlattım 0'a ve 1'e giderek yaklaşan orada bir olasılık değeri veren aktivasyon fonksiyonu sigmoidin kullanımı çift olasılıklı sinir ağlarında yaygındır.

# --- ÖNEMLİ ---
# YANİ SONUÇ OLARAK sigmoid fonksiyonu sayesinde her müşterinin bankadan ayrılıp ayrılmayacağını 0 ve 1 olarak bulmayacağız 0 ve 1 olma olasılıklarını bulacağız. yani her müşterinin bankadan ayrılıp ayrılmama olasılıklarını yüzde olarak görebileceğiz.
# Bu yüzden sigmoid fonksiyonunu kullanıyoruz.

## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Genelde "adam" optimizerı tercih edilir çünkü sthokastik Gradyan İnişini kullandığı için oldukça yüksek performans veren bir optimezerdır. Bu yöntem ile ağırlıkları güncelleyecektir.
# Tahminler ve Gerçek sonuçlar arasındaki kayıp fonksiyonunu bulmak için "loss" fonksiyonunu kullanıyoruz.
# 0 ve 1 kullanarak ikili sınıflandırma yapan bir sinir ağı ile çalıştığımız için loss = binary_crossentropy olmalıdır.
# Metrics, modelin performansını ölçmek ve değerlendirmek için kullanılır.
# Aynı anda birden fazla metrics se.ebileceğimiz için [] kullanılır. metrics genelde accuarry seçilir.
# Eğer ikili sınıfkandırmanın dışında çok daha çeşitli sınıfkandırmalar yapan bir ağ üzerinde çalışsaydık :

# ann.add(tf.keras.layers.Dense(units=3, activation='softmax'))
# ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# optimizer olarak genellikle "adam" gibi bir optimizasyon algoritması kullanılır. metrics ise genellikle doğruluk accuracy olarak ayarlanır, ancak isteğe bağlı olarak farklı metrikler de seçilebilir.


### Training the ANN on the Training set 

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
# "fit": Modelin eğitim verileri üzerinde eğitilmesini sağlayan yöntemdir.
# batch_size=32: genelde 32 seçilir. Mini-batch gradient descent yöntemiyle eğitim yapılırken kullanılacak batch boyutunu belirtir. Batch boyutu, her bir eğitim adımında kullanılacak örnek sayısını ifade eder. Küçük batch boyutları daha fazla bellek kullanımına neden olur ancak daha hızlı eğitim sağlayabilirken, büyük batch boyutları daha az bellek kullanır ancak eğitim süresi uzayabilir.
# epochs=100: Eğitim için kullanılacak epoch sayısını belirtir. Bir epoch, tüm eğitim verilerinin model tarafından bir kez geçirilmesini ifade eder. Epoch sayısı, modelin ne kadar süre boyunca eğitileceğini belirler.
# Epoch sayısını belirlerken, modelinizin doğruluğunu artırması ve overfitting (aşırı uyum) veya underfitting (yetersiz uyum) gibi problemleri minimize etmesi önemlidir.
# epoch değerini de veri setinin büyüklüğüne göre belirlemek iyi olacaktır.  Veri setiniz ne kadar büyükse, genellikle daha fazla epoch sayısı gerekebilir. Daha büyük veri setleri, modelin daha fazla örüntüyü öğrenmesi için daha fazla zaman gerektirir.
# Ancak yine de en iyi en optimize epoch değerini belirlemek yine deneme yanılmadan geçer.

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 888us/step - accuracy: 0.7941 - loss: 0.5044
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.7960 - loss: 0.4630
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.7960 - loss: 0.4447
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.7960 - loss: 0.4355
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.7964 - loss: 0.4292
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.8031 - loss: 0.4222
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.8101 - loss: 0.4147
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.8131 - loss: 0.4075
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - accuracy: 0.8154 - loss: 0.4003
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8163 - loss: 0.3928
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.8259 - loss: 0.3846
Epoch 12/100
250/25

In [21]:
# === KAYDETME HÜCRESİ — Eğitimin hemen ardından çalıştırın ===
import os, sys, json, pickle
from datetime import datetime
import tensorflow as tf

MODEL_VAR_NAME = None  

run_id = datetime.now().strftime("%Y%m%d-%H%M%S")
out_dir = f"artifacts_{run_id}"
os.makedirs(out_dir, exist_ok=True)

# 1) Keras modelini bul
_model = None
model_name = None
if MODEL_VAR_NAME and MODEL_VAR_NAME in globals():
    _model = globals()[MODEL_VAR_NAME]
    model_name = MODEL_VAR_NAME
else:
    for name, obj in list(globals().items()):
        if isinstance(obj, tf.keras.Model):
            _model = obj
            model_name = name
            break
if _model is None:
    raise RuntimeError("Eğitilmiş Keras modeli bulunamadı. MODEL_VAR_NAME = 'model' gibi ayarlayıp tekrar dene.")

# 2) Modeli kaydet (.keras önerilen format)
model_path = os.path.join(out_dir, "FatihAybsn.keras")
_model.save(model_path)

# 3) Model özetini yaz
with open(os.path.join(out_dir, "model_summary.txt"), "w", encoding="utf-8") as f:
    _model.summary(print_fn=lambda s: f.write(s + "\n"))

# 4) Eğitim geçmişi (history) varsa kaydet ve eğriyi PNG olarak çıkar
hist_obj = globals().get("history", None)
if hist_obj is not None and hasattr(hist_obj, "history"):
    history_dict = hist_obj.history
    with open(os.path.join(out_dir, "history.json"), "w", encoding="utf-8") as f:
        json.dump(history_dict, f, ensure_ascii=False, indent=2)
    try:
        import matplotlib.pyplot as plt
        plt.figure()
        if "loss" in history_dict: plt.plot(history_dict["loss"], label="loss")
        if "val_loss" in history_dict: plt.plot(history_dict["val_loss"], label="val_loss")
        if "accuracy" in history_dict: plt.plot(history_dict.get("accuracy", []), label="accuracy")
        if "val_accuracy" in history_dict: plt.plot(history_dict.get("val_accuracy", []), label="val_accuracy")
        plt.xlabel("Epoch"); plt.ylabel("Metric"); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(out_dir, "training_curves.png"), dpi=150)
        plt.close()
    except Exception as e:
        print("Eğitim grafiği kaydedilemedi:", e)

# 5) Ön-işleme nesneleri (varsa) kaydet
to_pickle = {
    "scaler": globals().get("sc") or globals().get("scaler") or globals().get("sc_X"),
    "column_transformer": globals().get("ct"),
    "onehot_geography": globals().get("ohe_geo") or globals().get("ohe") or globals().get("onehot"),
    "label_gender": globals().get("le_gender") or globals().get("le"),
}
for name, obj in to_pickle.items():
    if obj is not None:
        with open(os.path.join(out_dir, f"{name}.pkl"), "wb") as f:
            pickle.dump(obj, f)

# 6) Özellik isimleri (varsa) kaydet
feature_names = None
X_train = globals().get("X_train")
try:
    if X_train is not None and hasattr(X_train, "columns"):  # DataFrame ise
        feature_names = list(X_train.columns)
    else:
        ct = to_pickle.get("column_transformer")
        if ct is not None and hasattr(ct, "get_feature_names_out"):
            feature_names = list(ct.get_feature_names_out())
    if feature_names is not None:
        with open(os.path.join(out_dir, "feature_names.json"), "w", encoding="utf-8") as f:
            json.dump(feature_names, f, ensure_ascii=False, indent=2)
except Exception as e:
    print("Özellik isimleri kaydedilemedi:", e)

# 7) Ortam bilgisi ve kısa talimat
info = {
    "run_id": run_id,
    "model_variable": model_name,
    "python": sys.version,
    "tensorflow": tf.__version__,
}
try:
    import sklearn
    info["scikit_learn"] = sklearn.__version__
except Exception:
    pass

with open(os.path.join(out_dir, "README.txt"), "w", encoding="utf-8") as f:
    f.write(
        "Bu klasör, churn ANN eğitimi sonrası üretilen artefaktları içerir.\n"
        f"- Model: {os.path.basename(model_path)}\n"
        "- Ön-işleme nesneleri: *.pkl (varsa)\n"
        "- Eğitim geçmişi: history.json ve training_curves.png (varsa)\n"
        "- Özellik isimleri: feature_names.json (varsa)\n\n"
        "Yükleme örneği:\n"
        "import tensorflow as tf, pickle\n"
        f"model = tf.keras.models.load_model(r'{model_path}')\n"
        "# scaler/ct/encoders için: pickle.load(open('...pkl','rb'))\n"
    )

print(f"✓ Her şey '{out_dir}' klasörüne kaydedildi.")
print('Dosyalar:', *os.listdir(out_dir), sep="\n- ")


✓ Her şey 'artifacts_20251028-183617' klasörüne kaydedildi.
Dosyalar:
- column_transformer.pkl
- FatihAybsn.keras
- feature_names.json
- label_gender.pkl
- model_summary.txt
- README.txt
- scaler.pkl
